In [ ]:
# standard imports
import numpy as np
import xarray as xr
import rioxarray
import matplotlib.pyplot as plt

# load up the saved dataset
import pickle
# fp = '/Users/zachkeskinen/Documents/spicy-snow/tests/test_data/2_img_ds'
fp = '/Users/zachkeskinen/Documents/spicy-snow/data/10_img_dB.pkl'
with open(fp, 'rb') as f:
    ds = pickle.load(f)

# add module to path so python can import it. This is the directory with 
# the __init__.py file in it to let python know this is a module.
import sys
sys.path.append('/Users/zachkeskinen/Documents/spicy-snow/spicy_snow')

# import the function to test
from processing.snow_index import calc_delta_VV

# ds = ds.isel(time = slice(1, 3))